In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# importing dataset
train = pd.read_csv("train.csv")
train = train.set_index('ID')

In [3]:
test = pd.read_csv("test.csv")
test = test.set_index('ID')

In [4]:
# filling missing values
train = train.fillna(train['Credit_Product'].value_counts().index[0])
test= test.fillna(test['Credit_Product'].value_counts().index[0])

In [5]:
# encoding categorical values for training set
lb_make = LabelEncoder()

for i in train.columns:
    if train[i].dtype == 'object':
        train[i] = lb_make.fit_transform(train[i])


In [6]:
# encoding categorical values for testing set
lb_make = LabelEncoder()

for i in test.columns:
    if test[i].dtype == 'object':
        test[i] = lb_make.fit_transform(test[i])



In [7]:
# splitting data into training and testing
X = train.drop('Is_Lead', axis=1)
y = train['Is_Lead']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
# Standard python libraries
import os
import time
import logging
 
# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch
# Imports from our package
import lightautoml
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ImportanceCutoffSelector, ModelBasedImportanceEstimator
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task
from lightautoml.utils.profiler import Profiler
from lightautoml.automl.blend import WeightedBlender
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.report import ReportDeco

In [ ]:
#define all the parameters
N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TARGET_NAME = 'Is_Lead' # Target column name

In [ ]:
# Change profiling decorators settings
#By default, profiling decorators are turned off for speed and memory reduction. 
#If you want to see profiling report after using LAMA, you need to turn on the decorators using command below:
#create a Profile object to get the profile report
p = Profiler()
p.change_deco_settings({'enabled': True})



In [15]:
# from xgboost import XGBClassifier
# from sklearn.metrics import roc_auc_score
# xgb = XGBClassifier(eta=0.3,objective= 'binary:logistic',tree_method= 'gpu_hist',num_parallel_tree= 500,)
# xgb.fit(X_train, y_train)
# predictions = xgb.predict_proba(X_test)
# # print(type(model).__name__)
# print("    AUC_SCORE", roc_auc_score(y_test, predictions[:,1]))

In [39]:
result = xgb.predict_proba(test)

In [40]:
df=pd.read_csv("test.csv")
ID = df["ID"]
res = pd.DataFrame(ID)
res.columns = ["ID"]
abc= pd.DataFrame(result)
res["Is_Lead"] = abc[1]
res.head()
res.to_csv("submission.csv",index=False)
#res["Is_Lead"] = result[0]
# res.to_csv("submission.csv",index=False)